In [ ]:
#@title Install whisper
! pip install git+https://github.com/openai/whisper.git
! pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-cgf1wlld
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-cgf1wlld
  Resolved https://github.com/openai/whisper.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 KB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.5 MB/s eta 0:00:00
  Created wheel for ope

In [ ]:
#Imports
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import whisper
import torchaudio
import librosa
import glob

from tqdm.notebook import tqdm


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

Please upload cruise_audios.zip

In [ ]:
%%capture
!unzip /content/cruise_audios.zip

Archive:  /content/cruise_audios.zip
   creating: cruise_audios/
  inflating: cruise_audios/cruise001.wav  
  inflating: cruise_audios/cruise002.wav  
  inflating: cruise_audios/cruise003.wav  
  inflating: cruise_audios/cruise004.wav  
  inflating: cruise_audios/cruise005.wav  
  inflating: cruise_audios/cruise006.wav  
  inflating: cruise_audios/cruise007.wav  
  inflating: cruise_audios/cruise008.wav  
  inflating: cruise_audios/cruise009.wav  
  inflating: cruise_audios/cruise010.wav  
  inflating: cruise_audios/cruise011.wav  
  inflating: cruise_audios/cruise012.wav  
  inflating: cruise_audios/cruise013.wav  
  inflating: cruise_audios/cruise014.wav  
  inflating: cruise_audios/cruise015.wav  
  inflating: cruise_audios/cruise016.wav  
  inflating: cruise_audios/cruise017.wav  
  inflating: cruise_audios/cruise018.wav  
  inflating: cruise_audios/cruise019.wav  
  inflating: cruise_audios/cruise020.wav  
  inflating: cruise_audios/cruise021.wav  
  inflating: cruise_audios/cruis

In [ ]:
!ls -U ./cruise_audios | head -4

NotImplementedError: ignored

In [ ]:
class CruiseSet(torch.utils.data.Dataset):
    def __init__(self, dataset_path="/content/cruise_audios/", device=DEVICE):
        self.dataset_path=dataset_path
        self.device=device
        dataset=[]
        print("init cruise set. files found:")
        print(sorted(glob.glob(self.dataset_path+"*")))
        for wavfile in sorted(glob.glob(self.dataset_path+"*")):
            audio , s = librosa.load(wavfile, sr=16000) # Downsample 44.1kHz to 16kHz
            assert s == 16000
            audio=torch.from_numpy(audio)
            dataset.append(audio)
        self.dataset=dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        audio = self.dataset[item]
        audio = whisper.pad_or_trim(audio.flatten()).to(self.device) # why to I have to pad/trim
        mel = whisper.log_mel_spectrogram(audio)

        return (mel)

    def get_item_length(self, item):
        audio = self.dataset[item]
        duration=librosa.get_duration(y=audio, sr=16000)
        return(duration)

In [ ]:
dataset = CruiseSet()
loader = torch.utils.data.DataLoader(dataset, batch_size=16)

init cruise set. files found:
['/content/cruise_audios/cruise001.wav', '/content/cruise_audios/cruise002.wav', '/content/cruise_audios/cruise003.wav', '/content/cruise_audios/cruise004.wav', '/content/cruise_audios/cruise005.wav', '/content/cruise_audios/cruise006.wav', '/content/cruise_audios/cruise007.wav', '/content/cruise_audios/cruise008.wav', '/content/cruise_audios/cruise009.wav', '/content/cruise_audios/cruise010.wav', '/content/cruise_audios/cruise011.wav', '/content/cruise_audios/cruise012.wav', '/content/cruise_audios/cruise013.wav', '/content/cruise_audios/cruise014.wav', '/content/cruise_audios/cruise015.wav', '/content/cruise_audios/cruise016.wav', '/content/cruise_audios/cruise017.wav', '/content/cruise_audios/cruise018.wav', '/content/cruise_audios/cruise019.wav', '/content/cruise_audios/cruise020.wav', '/content/cruise_audios/cruise021.wav', '/content/cruise_audios/cruise022.wav', '/content/cruise_audios/cruise023.wav', '/content/cruise_audios/cruise024.wav', '/content

# Running inference on the dataset using a base Whisper model

The following will take a few minutes to transcribe all utterances in the dataset.

In [ ]:
model = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is English-only and has 71,825,408 parameters.


In [ ]:
# predict without timestamps for short-form transcription
options = whisper.DecodingOptions(language="en", without_timestamps=True)

In [ ]:
transcriptions = []

for mels in tqdm(loader):
    results = model.decode(mels, options)
    transcriptions.extend([result.text for result in results])

  0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
for i in range(len(transcriptions)):
    print(i+1, transcriptions[i])
#TODO, check all lines to make sure there are no big errors, in which case, correct them manually.
#1-100 checked
transcriptions[35]="Just layered through all of the movies, really. And even in, you know, and this one its"
transcriptions[61]="Power. Credibility."
transcriptions[65]="Crushes it. When she's on screen, you just"
transcriptions[69]="It's a love-letter to Paris and..."
transcriptions[74]="New Zealand. We could not have shot in Paris without all the help of the Parisians or"
transcriptions[80]="Wants that."
transcriptions[82]="Norway we're climbing the side of a mountain."
transcriptions[83]="And, you know, of course, London, we had the Footchase in London."
transcriptions[87]="And I know an audience will experience and we shot an Abu Dhabi and we had the UA military where we're able to"
transcriptions[95]="that always something that I'm trying to."
transcriptions[255]="To see my ability and make sure I want to make sure that you"
transcriptions[256]="Feel comfortable with my ability and you understand my ability."

1 proud of my friend and when I look at the movie, I'm going through it, I'm like, McEw, look, I mean...
2 He really is just enormously talented.
3 And I've always, it's always such a joy to work with them.
4 The Mission Impossible team. It's going to be Rebecca Ferguson who plays Elsa.
5 It really, the core team is going to be Simon, myself and Ving. That is the core team.
6 And that is the core mission of Possible Team, and Ilsa is...
7 I can't really say that Rebecca is, you have to see the movie.
8 But you'll see what happens with Rebecca's character.
9 that is the core mission team and Alec Baldwin as the secretary.
10 Each actor I think is very charismatic, very dynamic, and I'm interested in all their stories.
11 When I see them all together, I lean into that story.
12 That's what I want out of a movie.
13 that it's I want an audience to experience, you know.
14 That on screen is some of my favorite movies. It's like you have just to have a group of great actors just...
15 Hitti

In [ ]:
# Format wanted: {"audio_filepath": "/path/to/audio.wav", "text": "the transcription of the utterance", "duration": 23.147}
import json

# Data to be written
dataset_path="/content/cruise_audios/"
audio_filepaths=sorted(glob.glob(dataset_path+"*"))
durations = [dataset.get_item_length(i) for i in range(len(dataset))]
 
with open("manifest.json", "w") as outfile:
    for dico in [{ "audio_filepath": audio_filepaths[i], "text": transcriptions[i], "duration": durations[i] } for i in range(len(dataset))]:
        json.dump(dico, outfile)
        outfile.write("\n")

We can normalize the text using whisper's normalizer

In [ ]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

# data["hypothesis_clean"] = [normalizer(text) for text in data["hypothesis"]]
# data["reference_clean"] = [normalizer(text) for text in data["reference"]]
# data

In [ ]:
# import jiwer
# wer = jiwer.wer(list(data["reference_clean"]), list(data["hypothesis_clean"]))

# print(f"WER: {wer * 100:.2f} %")

WER: 4.26 %
